In [ ]:
import numpy as np

def kl_divergence(p, q, eps=1e-12):
    """
    KL(P||Q) = sum_i p_i * log(p_i / q_i)
    p, q: 1D array-like (probability vectors). 自動で正規化します。
    eps: 0除算・log(0)回避用の微小値
    """
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)

    if p.ndim != 1 or q.ndim != 1 or p.shape != q.shape:
        raise ValueError("p と q は同じ長さの 1 次元ベクトルである必要があります")

    if np.any(p < 0) or np.any(q < 0):
        raise ValueError("p と q は非負である必要があります")

    sp, sq = p.sum(), q.sum()
    if sp <= 0 or sq <= 0:
        raise ValueError("p と q の和は正である必要があります")

    # 正規化（入力が確率でない場合も許容）
    p = p / sp
    q = q / sq

    # クリップして数値安定化
    p = np.clip(p, eps, 1.0)
    q = np.clip(q, eps, 1.0)

    print(p)
    print(q)

    return float(np.sum(p * np.log(p / q)))

# 例
if __name__ == "__main__":
    p = [0.1, 0.7, 0.3]
    q = [1.2, 1.9, 1.2]
    print(kl_divergence(p, q))
    print(kl_divergence(q, p))


[0.09090909 0.63636364 0.27272727]
[0.33255891 0.33488218 0.33255891]
0.23654077908984145
[0.33255891 0.33488218 0.33255891]
[0.09090909 0.63636364 0.27272727]
0.2822844433629694


In [13]:
import numpy as np

def js_divergence(p, q, eps=1e-12, log_base=np.e):
    """
    JS(P||Q) = 0.5 * KL(P||M) + 0.5 * KL(Q||M), where M = 0.5*(P+Q)
    p, q: 1D array-like（確率ベクトル。自動で正規化）
    eps: 数値安定化用
    log_base: np.e（nat）または 2（bit）
    """
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)

    if p.ndim != 1 or q.ndim != 1 or p.shape != q.shape:
        raise ValueError("p と q は同じ長さの 1 次元ベクトルである必要があります")
    if np.any(p < 0) or np.any(q < 0):
        raise ValueError("p と q は非負である必要があります")

    # 正規化
    p = p / p.sum()
    q = q / q.sum()

    # 中間分布
    m = 0.5 * (p + q)

    # クリップ
    p = np.clip(p, eps, 1.0)
    q = np.clip(q, eps, 1.0)
    m = np.clip(m, eps, 1.0)

    # KL計算（底の変更）
    log = np.log if log_base == np.e else (lambda x: np.log(x) / np.log(log_base))
    kl_pm = np.sum(p * log(p / m))
    kl_qm = np.sum(q * log(q / m))

    return float(0.5 * (kl_pm + kl_qm))


# 例
if __name__ == "__main__":
    p = [0.2, 0.5, 0.3]
    q = [0.1, 0.7, 0.2]
    print(js_divergence(p, q))        # nat
    print(js_divergence(q, p))        # nat
    print(js_divergence(p, q, log_base=2))  # bit


0.021901178968493855
0.021901178968493855
0.03159672228746776


In [14]:
import numpy as np

def knn_predict(X_train, y_train, x, k=3):
    # 距離計算（ユークリッド）
    dists = np.linalg.norm(X_train - x, axis=1)
    idx = np.argsort(dists)[:k]
    # 分類（多数決）
    labels, counts = np.unique(y_train[idx], return_counts=True)
    return labels[np.argmax(counts)]

# 使用例
X = np.array([[1,1],[2,2],[3,3],[6,6]])
y = np.array([0,0,1,1])
print(knn_predict(X, y, np.array([2.5,2.5]), k=3))


0


In [17]:
import numpy as np

# =========================
# KD-Tree ノード
# =========================
class KDNode:
    def __init__(self, point, axis, left=None, right=None):
        self.point = point      # 分割点
        self.axis = axis        # 分割軸
        self.left = left
        self.right = right


# =========================
# KD-Tree 構築
# =========================
def build_kdtree(points, depth=0):
    if len(points) == 0:
        return None

    k = points.shape[1]            # 次元数
    axis = depth % k               # 分割軸
    points = points[points[:, axis].argsort()]
    mid = len(points) // 2

    return KDNode(
        point=points[mid],
        axis=axis,
        left=build_kdtree(points[:mid], depth + 1),
        right=build_kdtree(points[mid + 1:], depth + 1)
    )


# =========================
# 最近傍探索（1-NN）
# =========================
def nearest_neighbor(node, target, best=None):
    if node is None:
        return best

    dist = np.linalg.norm(node.point - target)
    if best is None or dist < best[1]:
        best = (node.point, dist)

    axis = node.axis
    diff = target[axis] - node.point[axis]

    # 近い側を先に探索
    near, far = (node.left, node.right) if diff < 0 else (node.right, node.left)
    best = nearest_neighbor(near, target, best)

    # 分割面を越える可能性があれば反対側も探索
    if abs(diff) < best[1]:
        best = nearest_neighbor(far, target, best)

    return best


# =========================
# 使用例
# =========================
points = np.array([
    [2, 3],
    [5, 4],
    [9, 6],
    [4, 7],
    [8, 1],
    [7, 2],
], dtype=float)

tree = build_kdtree(points)

query = np.array([3, 4], dtype=float)
nearest_point, distance = nearest_neighbor(tree, query)

print("Nearest:", nearest_point)
print("Distance:", distance)


Nearest: [2. 3.]
Distance: 1.4142135623730951


In [27]:
import numpy as np

def kl_hist(p_samples, q_samples, bins=30, eps=1e-12):
    """
    サンプル数が異なる2つのデータ列から、
    同じビンでヒストグラム確率分布を作って KL(P||Q) を計算する最小実装。
    """
    # 両者をカバーする共通レンジ（同じ確率空間に揃える）
    lo = min(np.min(p_samples), np.min(q_samples))
    hi = max(np.max(p_samples), np.max(q_samples))

    print(lo, hi)

    p_hist, edges = np.histogram(p_samples, bins=bins, range=(lo, hi))
    q_hist, _     = np.histogram(q_samples, bins=edges)

    print(p_hist)
    print(q_hist)

    # 確率に正規化（データ数が違ってもOK）
    P = p_hist / p_hist.sum()
    Q = q_hist / q_hist.sum()


    # 0確率対策（KL発散回避のため）
    P = np.clip(P, eps, 1.0)
    Q = np.clip(Q, eps, 1.0)
    P /= P.sum()
    Q /= Q.sum()

    print(f"{P=}")
    print(f"{Q=}")
    return float(np.sum(P * np.log(P / Q)))

if __name__ == "__main__":
    rng = np.random.default_rng(0)

    # サンプル数が異なる例
    # P_samples = rng.normal(loc=0.0, scale=1.0, size=1000)
    # Q_samples = rng.normal(loc=0.5, scale=1.2, size=120)
    P_samples = np.array([0.1, 0.4, 0.5, 0.6, 0.8])
    Q_samples = np.array([0.2, 0.3, 0.5, 0.7])

    print(P_samples)
    print(Q_samples)

    kl = kl_hist(P_samples, Q_samples, bins=40)
    print("KL(P||Q) =", kl)


[0.1 0.4 0.5 0.6 0.8]
[0.2 0.3 0.5 0.7]
0.1 0.8
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0]
P=array([2.e-01, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12,
       1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12,
       1.e-12, 2.e-01, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 2.e-01, 1.e-12,
       1.e-12, 1.e-12, 1.e-12, 1.e-12, 2.e-01, 1.e-12, 1.e-12, 1.e-12,
       1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 1.e-12, 2.e-01])
Q=array([1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 2.5e-01, 1.0e-12,
       1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 2.5e-01, 1.0e-12, 1.0e-12,
       1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12,
       1.0e-12, 2.5e-01, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12,
       1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 2.5e-01,
       1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12, 1.0e-12])
KL(P||Q) = 20.772637851